In [5]:
import pandas as pd
path = '/content/train.csv'
df = pd.read_csv(path)
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df = df.dropna()
X = df.drop('label', axis=1)
y = df['label']

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

# Data Preprocessing

In [8]:
voc_size = 5000
messages = X.copy()
messages.reset_index(inplace=True)

In [9]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [12]:
one_hot_rep = []
for words in corpus:
  one_hot_rep.append(one_hot(words,voc_size))
one_hot_rep

[[611, 2818, 1222, 232, 3887, 4175, 4687, 2835, 3906, 2641],
 [1028, 201, 4564, 2874, 305, 3473, 3704],
 [3324, 2691, 2519, 1349],
 [2380, 4597, 637, 1683, 3501, 3337],
 [3598, 305, 2085, 1919, 2474, 1617, 305, 2544, 1029, 4577],
 [2277,
  3509,
  3966,
  337,
  2976,
  3189,
  1702,
  3254,
  2698,
  4025,
  4716,
  432,
  1368,
  2993,
  3704],
 [2644, 996, 2326, 1080, 4406, 2791, 3906, 1422, 3967, 3368, 96],
 [3569, 4505, 4895, 3970, 4950, 1739, 3189, 3380, 3967, 3368, 96],
 [3541, 4797, 2956, 3009, 4359, 4558, 4732, 3233, 3189, 510],
 [4136, 3383, 2940, 3484, 1503, 3701, 1698, 1049],
 [2085, 2980, 4629, 4123, 1930, 2274, 1449, 4057, 190, 756, 1675],
 [1683, 4881, 3887, 4558, 3189, 4950],
 [864, 4994, 1827, 1786, 2553, 3946, 3308, 3213, 4981],
 [850, 1402, 3829, 3629, 4344, 2267, 1440, 3967, 3368, 96],
 [489, 4379, 1913, 2146, 3111, 3967, 3368, 96],
 [4739, 126, 852, 1697, 1276, 3172, 1359, 421, 1539, 661],
 [1436, 2906, 201],
 [1616, 4449, 2763, 2628, 3189, 3869, 1717, 3704],
 [380

In [ ]:
sentence_length = 20
embedded_doc = pad_sequences(one_hot_rep, padding='pre', maxlen=sentence_length)
print(embedded_doc)

In [14]:
embedded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  611,
       2818, 1222,  232, 3887, 4175, 4687, 2835, 3906, 2641], dtype=int32)

# Model

In [43]:
model = Sequential()
model.add(Embedding(voc_size, 40, input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               56400     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [22]:
import numpy as np
X_final = np.array(embedded_doc)
y_final = np.array(y)
X_final.shape

(18285, 20)

In [23]:
y_final.shape

(18285,)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2)

In [44]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
458/458 [==============================] - 16s 24ms/step - loss: 0.2610 - accuracy: 0.4361
Epoch 2/10
458/458 [==============================] - 12s 27ms/step - loss: 0.1323 - accuracy: 0.4361
Epoch 3/10
458/458 [==============================] - 12s 26ms/step - loss: 0.0918 - accuracy: 0.4361
Epoch 4/10
458/458 [==============================] - 9s 19ms/step - loss: 0.0609 - accuracy: 0.4361
Epoch 5/10
458/458 [==============================] - 11s 24ms/step - loss: 0.0367 - accuracy: 0.4361
Epoch 6/10
458/458 [==============================] - 11s 23ms/step - loss: 0.0249 - accuracy: 0.4361
Epoch 7/10
458/458 [==============================] - 9s 20ms/step - loss: 0.0168 - accuracy: 0.4361
Epoch 8/10
458/458 [==============================] - 14s 31ms/step - loss: 0.0108 - accuracy: 0.4361
Epoch 9/10
458/458 [==============================] - 19s 42ms/step - loss: 0.0107 - accuracy: 0.4361
Epoch 10/10
458/458 [==============================] - 15s 34ms/step - loss: 0.0052 

In [49]:
from sklearn.metrics import accuracy_score
threshold = 0.5
y_pred = model.predict(X_test)
y_pred_binary = (y_pred >= threshold).astype(int)
accuracy = accuracy_score(y_pred_binary,y_test)
print(accuracy)

0.4222039923434509


# Adding Dropout

In [53]:
from tensorflow.keras.layers import Dropout
model = Sequential()
model.add(Embedding(voc_size, 40, input_length=sentence_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [54]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_1 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_7 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [55]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
458/458 [==============================] - 16s 29ms/step - loss: 0.2669 - accuracy: 0.8816
Epoch 2/10
458/458 [==============================] - 18s 38ms/step - loss: 0.1436 - accuracy: 0.9437
Epoch 3/10
458/458 [==============================] - 11s 24ms/step - loss: 0.1065 - accuracy: 0.9597
Epoch 4/10
458/458 [==============================] - 12s 26ms/step - loss: 0.0760 - accuracy: 0.9722
Epoch 5/10
458/458 [==============================] - 10s 22ms/step - loss: 0.0579 - accuracy: 0.9802
Epoch 6/10
458/458 [==============================] - 11s 24ms/step - loss: 0.0422 - accuracy: 0.9865
Epoch 7/10
458/458 [==============================] - 11s 24ms/step - loss: 0.0304 - accuracy: 0.9895
Epoch 8/10
458/458 [==============================] - 9s 21ms/step - loss: 0.0266 - accuracy: 0.9907
Epoch 9/10
458/458 [==============================] - 11s 24ms/step - loss: 0.0210 - accuracy: 0.9930
Epoch 10/10
458/458 [==============================] - 11s 24ms/step - loss: 0.0193

In [56]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
y_pred = (y_pred >= threshold).astype(int)
accuracy = accuracy_score(y_pred,y_test)
print(accuracy)

115/115 [==============================] - 2s 7ms/step
0.9237079573420837
